# Estructura de la programación modular

En este _notebook_ discutiremos algunas de las limitaciones que conlleva escribir un programa en un solo archivo de texto, así como las soluciones que ofrece la llamada [_programación modular_](https://en.wikipedia.org/wiki/Modular_programming).

## _Escribir un programa en un solo archivo puede ser problemático_

Como vimos en el _notebook_ [`1.4-Variables_constantes_y_funciones.ipynb`](./1.4-Variables_constantes_y_funciones.ipynb), podemos definir nuestras propias funciones para ejecutar series de instrucciones específicas siempre que resulte conveniente, incluso modificando valores y/o variables si es necesario. Gran parte del poder de la programación yace en esta capacidad de poder crear nuestras propias funciones; sin embargo, con lo que hemos visto hasta el momento, esto también nos podría generar los siguientes problemas e inconvenientes:
* si queremos usar algunas funciones para muchos programas distintos, tenemos que copiar y pegar las definiciones a cada uno de los archivos en que las queramos usar;
* si, posteriormente, queremos modificar a una de las funciones en todos los programas en que la utilizamos, tenemos que modificar su definición en cada uno de los archivos en los que se define.

Más aún, si queremos usar funciones que otra persona ha definido, tenemos que acceder al código fuente (esto es, si el código es abierto) y copiar y pegar las definiciones en todos los archivos en que las queramos usar, y sucedería algo similar si quisiéramos compartir nuestras propias funciones con otras personas.

## Programación modular

Todos los problemas mencionados anteriormente se podrían resolver si todas las funciones pudieran definirse en **un sólo archivo** (de código abierto), que pudiéramos _cargar_ en cada uno de los programas que las requieran. Afortunadamente, ¡esto es posible!

Lo anterior es un ejemplo de **programación modular** donde, _en vez de escribir todo un programa_ (que puede tener una longitud y complejidad arbitraria) _en un solo archivo_, _lo separamos en varios archivos diferentes_ y, cada vez que una parte del código escrito en un archivo $a$ se requiera en un archivo $b$, podamos dar la instrucción de que se ejecute el código del archivo $a$ con algún comando en el archivo $b$.

### `include`

En Julia, la función `include` sirve para ejecutar el código de otro archivo dentro del archivo donde se utilice esta función. Por ejemplo, para ejecutar el código (de Julia) de un archivo `a.jl` en otro archivo `b.jl`, escribiríamos el comando

```include("a.jl")```

en alguna línea del archivo `b.jl`. En esencia, esto _equivale a reemplazar la línea anterior del archivo `b.jl` con todo el contenido del archivo `a.jl`_.

Algunas consideraciones importantes al usar `include` son las siguientes.
1. Como estamos trabajando en el paradigma imperativo, el código se ejecuta línea por línea de arriba hacia abajo.
1. Si escribimos el comando `include("a.jl")` varias veces en un archivo, entonces todos los contenidos de `a.jl` se ejecutarán cada vez que aparezca `include("a.jl")` en el código.
1. Si el argumento de `include` no incluye una dirección absoluta, se interpreta en relación a la dirección del archivo donde se utiliza esta función; es decir si, por ejemplo, `b.jl` está en la carpeta `~/MisArchivos/` (y, por ende, su dirección absoluta es `~/MisArchivos/b.jl`), entonces el comando `include("a.jl")` en `b.jl` intentará ejecutar el archivo `~/MisArchivos/a.jl`.

No tener presentes las observaciones anteriores podría traernos problemas. Por ejemplo, supongamos que en `a.jl` se definen variables y/o funciones. Si alguna de ellas se utiliza en el código `b.jl` _antes_ del comando `include("a.jl")` entonces, por 1, obtendremos un error de que aún no ha sido definida. Por otro lado, si usamos `include("a.jl")` varias veces entonces, por 2, a las variables definidas en `a.jl` se les volverá a asignar el valor que tienen en ese archivo en cada línea donde se ejecute `include("a.jl")`, lo cual puede no ser deseable. Más aún, si se incluye el código de dos archivos `a.jl` y `c.jl` en un archivo `b.jl`, entonces los códigos de los tres archivos deben ser compatibles, es decir, que no se utilicen los mismos nombres para variables (o funciones) que tengan asignaciones (o definiciones) diferentes para evitar colisiones de nombres o reasignaciones indeseadas.

Algunos usos de la programación modular son los siguientes.
* Definir todas las constantes que se utilicen en un programa en un archivo apartado del resto del programa.
* Definir funciones que cumplan un objetivo específico y puedan ser utilizadas en una gran variedad de programas diferentes en un archivo aparte.
* Partir un proyecto grande en varios subproyectos, permitiendo mayor claridad, independencia y organización.
* Mover el código de una "interfaz" a un archivo apartado para que sea más fácil de refactorizar, es decir, de _reordenar su código "interno" sin cambiar su comportamiento "externo"_.

## Módulos

Una forma de hacer programación modular en Julia más sofisticada que usar `include` es usando módulos. Los **módulos** nos permiten organizar un código en unidades coherentes e independientes. Algunas de las ventajas de usar módulos incluyen:
* poder incluir partes del código de un módulo _selectivamente_, es decir, sin necesidad de incluir _todo_ el código del módulo;
* los módulos pueden ser _precompilados_ (es decir, convertidos a lenguaje de máquina) de antemano, ahorrándonos tiempo de cómputo.

### Módulos estándar en Julia

Julia tiene tres módulos estándar:

* `Core`, que contiene todas las definiciones necesarias para que Julia tenga una funcionalidad mínima,
* `Base`, que contiene toda la funcionalidad básica de Julia, y
* `Main`, que es el módulo más alto en la jerarquía de módulos.

Para poder funcionar de manera correcta -incluyendo la funcionalidad básica y la capacidad de cargar otros módulos-, _implícitamente_, Julia **siempre**:

* se inicializa dentro del módulo `Main` y
* carga los módulos `Core` y `Base`. 

### Creación de módulos

Para crear un módulo, utilizamos una sintáxis como la siguiente:

In [ ]:
module MiMódulo                           # Inicializamos el módulo.

export miFunción1, miFunción2             # Especificamos qué cosas definidas
                                          # dentro del módulo podrán ser
                                          # importadas en otros archivos.

    function miFunción1()                 # Definimos las cosas que exporta-
                                          # remos.
        print("Acabas de llamar a miFunción1.")
    end

    function miFunción2()

        print("Acabas de llamar a miFunción2.")
    end

    function miFunción3()

        print("Acabas de llamar a miFunción3.")
    end
end                                        # Finalizamos el módulo.

Notamos que la declaración $\color{green}{\texttt{module}}$, que inicia un bloque de código que Julia convertirá en un módulo y declara el nombre del módulo, va al inicio del archivo. Por conveniencia, justo después viene la declaración `export`, donde se nombran todas las variables, constantes, funciones, etcétera que queremos que puedan ser importadas en otros archivos, antes de que éstas sean definidas.

### Carga de módulos

#### `using`

Una forma de cargar todos los contenidos de exportados por un módulo es utilizando la sintáxis `using NombreDelMódulo`. Por ejemplo, en el _notebook_ [`1.7-Ciclos.ipynb`](./1.7-Ciclos.ipynb), importamos todos los contenidos del módulo [`ThinkJulia`](https://github.com/BenLauwens/ThinkJulia.jl/blob/master/src/ThinkJulia.jl) con el siguiente comando:

In [ ]:
using ThinkJulia

Después de haber ejecutado este comando, podremos llamar a todos los nombres que aparecen en la declaración `export` del módulo correspondiente sin problema alguno. Un ejemplo con el módulo que definimos anteriormente es:

In [ ]:
using Main.MiMódulo # Importamos el módulo que definimos anteriormente

In [ ]:
miFunción2() # Lo que enlistamos en la declaración `export` funciona bien

In [ ]:
miFunción3() # No funciona porque no lo incluimos en la declaración `export`

Si algo se definió dentro del módulo `NombreDelMódulo` pero no se incluyó su nombre en la declaración `export`, podemos llamarlo si lo precedemos de la sintáxis `NombreDelMódulo.`:

In [ ]:
Main.MiMódulo.miFunción3()

Podemos cargar varios módulos con un solo comando `using` separando sus nombres con una coma, es decir, como en `using Módulo1, Módulo2, Módulo3`.

Si sólo queremos cargar algunos de los contenidos de un módulo a nuestro código, podemos utilizar la sintáxis `using NombreDelMódulo: Nombre1, Nombre2, Nombre3`. De esta manera, no tenemos que cargar _todo_ el módulo, sino sólo las partes que utilizaremos.

**Nota** A veces resulta deseable cargar un módulo con un nombre distinto al original. La mejor forma de hacer esto es definir una constante con el nombre que queremos para el módulo y asignarle como valor el nombre original del módulo. Por ejemplo:

In [ ]:
const MM = Main.MiMódulo #Definir el nombre como constante optimiza su sustitución.

In [ ]:
MM.miFunción3() #Resulta más sencillo escribir esto que 'Main.MiMódulo.miFunción3()'.

#### `import`

También se pueden cargar los contenidos de un módulo con la sintáxis `import NombreDelMódulo`. Sin embargo, este comando ignorará la declaración `export` del módulo `NombreDelMódulo`, por lo que tendremos que escribir `NombreDelMódulo.` antes de cualquier nombre que haya sido definido en ese módulo para poder usarlo.

## Paquetes

Para poder compartir nuestros módulos con otras personas de manera sencilla, podemos usarlos para formar **paquetes**. A grosso modo, un _paquete_ de Julia es un módulo con toda la estructura necesaria para que pueda ser _instalado_ en una computadora.

### `Pkg`

En Julia, los paquetes se instalan, actualizan y desinstalan utilizando el administrador de paquetes `Pkg`. Para entrar al modo de administración de paquetes en un REPL de Julia, basta con presionar la tecla `]`; ¡esto es justo lo que te pedimos hacer en las [instrucciones para instalar IJulia y Pluto del repositorio del curso](https://github.com/dabnciencias/AC/)! Una vez que Julia está en modo de administración de paquetes -y el _prompt_ de la terminal haya cambiado a algo como `pkg>`-, la sintáxis para instalar, actualizar y desinstalar paquetes es
* `add Paquete`,
* `update Paquete`, y
* `rm Paquete`,

respectivamente. Como Jupyter no es propiamente un REPL, tendremos que escribir `] ` (con un espacio) antes de los comandos anteriores para que Julia sepa que debe entrar al modo de administración de paquetes primero: 

In [ ]:
] add LinearAlgebra

In [ ]:
] update LinearAlgebra

In [ ]:
] rm ThinkJulia

¡Oh, no, acabamos de desinstalar el paquete ThinkJulia con el que dibujábamos con tortuguitas! Rápido, ¡reinstalémoslo!

In [ ]:
] add ThinkJulia

A diferencia de los paquetes anteriores, parece que `ThinkJulia` no está 'registrado' (lo que sea que eso signifique). Para poder instalar un paquete no registrado, debemos especificar el URL en el que se encuentra. En el caso de ThinkJulia, este paquete se encuentra disponible en un repositorio de GitHub llamado [ThinkJulia.jl](https://github.com/BenLauwens/ThinkJulia.jl).

In [ ]:
] add https://github.com/BenLauwens/ThinkJulia.jl

## Resumen

Existen varias limitaciones que surgen de querer escribir todo el código de un programa en un solo archivo, que se resuelven a través de la _programación modular_ -esto es, la separación de código en unidades independientes que se pueden integrar unas con otras. La programación modular ofrece muchos beneficios relacionados con la compacidad, claridad, reutilizabilidad y distributividad de código, por lo que es importante entender sus principios.

Julia ofrece formas de cargar módulos ajenos y crear módulos propios, así como instalar y compartir módulos a través de internet ofrecidos en forma de _paquetes_ usando el administrador de paquetes `Pkg`.

## Recursos complementarios
* Página de Wikipedia de [programación modular](https://en.wikipedia.org/wiki/Modular_programming).
* Manual de [Code Loading](https://docs.julialang.org/en/v1/manual/code-loading/) de Julia.
* Manual de [Submódulos](https://docs.julialang.org/en/v1/manual/modules/#Submodules-and-relative-paths) de Julia.
* [Preguntas frecuentes sobre Módulos y Paquetes](https://docs.julialang.org/en/v1/manual/faq/#Packages-and-Modules) en Julia.
* Documentación del [administrador de paquetes `Pkg`](https://docs.julialang.org/en/v1/stdlib/Pkg/) de Julia .